# Introduction to Geopandas, Matplotlib, Contextily and GIS

### In this lesson, we will:

 - Learn about different types of geographic data: point data and polygon data
 - Learn about Census Geographies or the levels at which data can be grouped and analyzed 
 - Continue to practice dataframe wrangling and merging, this time specifically with merging dataframes to shapefiles
 - Practice mapping datasets at the county, census tract, and point level 
 - Make multiple map layers and combien them in one visualization using contextily for baselayers and labels, and adding polygon and point data over them

# GIS Data Types

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.arcgis.com/sharing/rest/content/items/58593bfc09c741a0b67dd81c22a914e3/info/thumbnail/ago_downloaded.png/?w=800")

## Point Data: Individual Cases of Something

In [ ]:
Image(url= "https://tpwd.texas.gov/gis/data/baselayers/citypts/image")

In [ ]:
Image(url= "https://www.esri.com/arcgis-blog/wp-content/uploads/2018/02/05-fig-5-6-v2.png")

## Polygon Data: Shape Files

In [ ]:
Image(url= "https://www.sheriffstx.org/templates/sheriffstx.org/images/tx_county_map.gif")

## Chloropleth Maps

In [ ]:
Image(url= "https://public.tableau.com/static/images/Te/TexasCounts2020CensusLandscapeMap/Sheet1/1_rss.png")

# The Hierarchy of Census Geographies

<img src="../images/census_geography_hierarchy_labelled.png" width=700 height=700 />

## FIPS Geo-Identifier Codes

<img src="../images/fips_code_explainer.png" width=700 height=700 />

Let's begin by loading in all the relevant libraries we're going to use 

In [ ]:
import pandas as pd
import csv
import numpy as np
import math
from IPython.display import Image, display
import geopandas as gpd

import matplotlib.pyplot as plt

%matplotlib inline
from shapely.geometry import Point, Polygon
from geopandas import datasets, GeoDataFrame, read_file

#importing contextily library which adds raster basemaps from open street maps, experimenting with adding these
#to our maps, playing with zoom and other functions to see what baselayer detail we can get

import contextily as cx

### We're going to download the County and Census Tract Shapefiles from the US Census Bureau's [TIGER/LIne Database](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)

In [ ]:
#load in the Tiger/Line Place shapefile for the state of Texas, read it using geopandas
ct = "../data/tl_2021_48_tract/tl_2021_48_tract.shp"
data = gpd.read_file(ct)

In [ ]:
data

In [ ]:
#Check the data types of each column to help with merging later on
data.dtypes

We can see that the GEOID column, where out fips code for merging is stored, is an object/string data type. We're going to need to convert it to int64 dtype for merging.

In [ ]:
#Write a function that will convert the 'str' fips code to 'int64' for merging, we use int64 because it can handle
#NaN data whereas 'int' cannot

def fips_2_int(df,fips_col,new_col):
        df[new_col] = [int(x) if type(x)==str else 
                       (np.nan if math.isnan(x)==True else(int(x) if type(x)==float else int(x))) 
                       for x in df[fips_col]]
        df[new_col] = df[new_col].astype('Int64')
        return df

In [ ]:
fips_2_int(data,'GEOID','FIPS_int64')

Our shapefile is reayd for merging, however, is it ready for subsetting and grouping at the county-level moving forward?

In [ ]:
#Use list comprehension to take the geoid column, which has the full 11 digit fips code down to the county level,
#taking all but the last 5 of the 11 digits

county_fips = []

for x in data['GEOID']:
    county_fips.append(x[:5])
    
data['county_fips'] = county_fips

In [ ]:
data.head()

__Exercise #1__ -- How could you have arrived at the same output using the STATEFP and COUNTYFP columns? If you have the time, practice creating a new column with the 5-digit county fips from these two columns below.

In [ ]:
#Exercise 1 code




In [ ]:
#Check the crs or projection type for this census tract shapefile
data.crs

# CDC Social Vulnerability Index Data - 2018

In [ ]:
#Read in the cdc social vulnerability data already subset for Texas Census Tracts 
svi = pd.read_csv('../data/svi_texas_census_tracts.csv')

In [ ]:
svi

In [ ]:
#Check the number of columns and what variables they contain
svi.columns

There are way too many variables in there that may not be useful for our analysis and mapping, go through the codebook to see what variable names are associated with what data, and decide what we want to keep

Since we are keeping a lot of data, we can make different lists of variables we want to keep grouped by variable type. That way we could make separare dataframes for each variable type, or concatenate them together.

In [ ]:
dem_var = ['FIPS','AREA_SQMI','E_TOTPOP','E_HU','E_HH','E_POV','EP_POV','E_UNEMP','EP_UNEMP',
                     'E_PCI','E_NOHSDP','EP_NOHSDP','E_AGE65','EP_AGE65','E_AGE17','EP_AGE17','E_DISABL',
                     'EP_DISABL',"E_SNGPNT",'EP_SNGPNT','E_MINRTY','EP_MINRTY','E_LIMENG','EP_LIMENG',
                  'E_MUNIT','EP_MUNIT','E_MOBILE','EP_MOBILE','E_CROWD','EP_CROWD','E_NOVEH','EP_NOVEH','E_GROUPQ',
                  'EP_GROUPQ']

vul_index_var = ['RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES','F_THEME1','F_THEME2',
                'F_THEME3','F_THEME4','F_TOTAL']

adjunct_var = ['E_UNINSUR','EP_UNINSUR','E_DAYPOP'] 

In [ ]:
#Add or concatenate these variable groups together
svi_var = dem_var+vul_index_var+adjunct_var

In [ ]:
#USe the filter function to only keep the columns/variables of interest
svi_clean = svi.filter(svi_var,axis=1)

In [ ]:
svi_clean

In [ ]:
svi_clean.dtypes

In [ ]:
#svi_geo_tracts = svi_clean.merge(data, left_on='FIPS', right_on='FIPS_int64', how='left')
svi_geo_tracts = svi_clean.merge(data, left_on='FIPS', right_on='FIPS_int64', how='outer')

The documentation for the SVI tells us that all missing data is coded as "-999", not NaN like is necessary for python. This will throw off all the analyses and maps we make from this dataset if we don't chnage it. 

In [ ]:
#Subset the df to include only values with -999
svi_geo_tracts[svi_geo_tracts["F_TOTAL"]==-999]

In [ ]:
#Went through svi_geo_tracts df to turn -999 values (what svi used to designate missing data) to np.nan to add them to 
#the missing data category.Otherwise they skew the quantiles for all ther following Cloropleth Maps. 

svi_geo_tracts = svi_geo_tracts.replace(-999, np.NaN)

In [ ]:
#Check for -999 values to make sure we replaced all of them
svi_geo_tracts[svi_geo_tracts["F_TOTAL"]==-999]

In [ ]:
#Take the df and turn it into a geo-df using the geopandas GeoDataFrame function, telling the function what the 
#crs is and telling it to take the df's geometry column as the geometry to be mapped
svi_gdf = gpd.GeoDataFrame(svi_geo_tracts, crs="EPSG:4269", geometry='geometry')

In [ ]:
#Use the plot function without any arguments to make the simplest plot of our data, this will take only the 
#data stored in the geometry column, strictly the 
svi_gdf.plot()

The plot function has multiple arguments we can use to add information to our map and customize it.

Function Arguments:
- __column__: choose which data column/variable you want to map onto your chosen geography
- __figsize__: set the size of the reuslting map
- __cmap__: tells matplotlib to make a cloropleth map and tells it what color gradient set to use to show variation
- __scheme__: sets how color changes across the gradient for the chosen variable are represented. Can be by quantiles in the data, by natural breaks, or by manually set data ranges. 
- __edgecolor__: sets the color of the outline of the geometry you are mapping, both polygon and point
- __alpha__: sets the level of transparency for your map layer, useful if you have basemap of roads and buildings underneath, or using multiple layers to allow data to be seen at multiple levels. Lower number = more transparent.
- __legend__: adds a legend to your cloropleth map telling what variable data rnage is associated with each color on the map

In [ ]:
#Make a cloropleth map of the SVI Index's measure of total vulnerability percentage

ax = svi_gdf.plot(column='RPL_THEMES',
                   figsize=(15, 10),
                   cmap='OrRd',
                   scheme='quantiles',
                   edgecolor='black',
                        alpha=0.7,
                   legend=True
                 )

__Exercise #2__ -- Try making a cloropleth map using a different column and using the "Blues" or "RdPu" color gradient sets below.

In [ ]:
#Exercise 2 code




The last map had a lot of information and gives a rough idea of where vulnerability is high in the state. However, it is still pretty messy, which missing census tracts, a graphing axis which isn't present in maps, and no title.

We can deal with these issue by using the __missing_kwds, .suptitle, .title, and .axis__ arguments/functions.

In [ ]:
ax = svi_gdf.plot(column='RPL_THEMES',
                        zorder=2,
                   figsize=(15, 10),
                   cmap='OrRd',
                   scheme='quantiles',
                   edgecolor='black',
                        alpha=0.7,
                   legend=True,
                   missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
                       "alpha":0.4,
        "hatch": "///",
        "label": "Missing values",
    })

plt.suptitle('Social Vulnerability Index for State of Texas',y=.94, fontsize=25)
plt.title('Total Vulnerability Themes Percentile by Census Tract')

ax.axis('off')

What do we do if we want to zoom in and look at a specific area on the map in more detail? Say an MSA or county?

To do this, we just need to use dataframe subsetting/slicing methods we have learned before and use them on the fips code identifiers.

In [ ]:
five_counties = svi_gdf[(svi_gdf['county_fips']=='48453')|(svi_gdf['county_fips']=='48021')|
                        (svi_gdf['county_fips']=='48055')|(svi_gdf['county_fips']=='48209')|
                        (svi_gdf['county_fips']=='48491')]

#travis = 48453
#bastrop = 48021
#caldwell = 48055
#hays = 48209
#williamson = 48491

#travis_county.boundary.plot()
five_counties.plot(column='RPL_THEMES',figsize=(15, 10),cmap='OrRd',scheme='quantiles', edgecolor='black',
                   legend=True, missing_kwds={"color": "lightgrey",
        "edgecolor": "red",
                       "alpha":0.4,
        "hatch": "///",
        "label": "Missing values",})

So now we only see the Central Texas /Austin MSA 5-County Area. However, we don't know where the census tracts fall in terms of each county. What if we wanted to add county boundaries so we can see where those breaks are?

Read in county shapefiles to overlay as a boundary layer over the svi data at the census tract level, highlight separations between five counties.

In [ ]:
county_shp = "../data/tl_2021_us_county/tl_2021_us_county.shp"
data_2 = gpd.read_file(county_shp)

In [ ]:
fips_2_int(data_2,'GEOID','FIPS_int64')

In [ ]:
data_2.plot()

In [ ]:
five_county_shp = data_2[(data_2["FIPS_int64"]==48453)|(data_2["FIPS_int64"]==48021)|
                        (data_2["FIPS_int64"]==48055)|(data_2["FIPS_int64"]==48209)|
                        (data_2["FIPS_int64"]==48491)]

In [ ]:
five_county_shp = five_county_shp.to_crs(epsg=3857)

In [ ]:
five_county_shp.plot(facecolor='none',edgecolor='black',linewidth=3)

Let's add this 5-county boundary layer to the SVI map we made above.

In [ ]:
plt.figure()

five_counties = five_counties.to_crs(epsg=3857)
five_county_shp = five_county_shp.to_crs(epsg=3857)

base = five_counties.plot(column='F_TOTAL',figsize=(20, 15),cmap='OrRd',scheme='natural_breaks', edgecolor='black',
                          legend=True,
                         missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
                       "alpha":0.2,
        "hatch": "///",
        "label": "Missing values"})

full = five_county_shp.plot(ax=base,facecolor='none',edgecolor='black',linewidth=3)

full.set_axis_off()
plt.suptitle('Social Vulnerability Index for Central Texas Five-County Region',y=.93, fontsize=25)
plt.title('Total Vulnerability Flags by Census Tract')

#plt.savefig('../images/ct5_svi_total_flags.jpg')

# Community Partners Point Data

We played around with and successfully mapped polygon data of Social Vulnerability above. What if we wanted to see how well medical service providers partnered with Dell Medical School are meeting their mission of treating vulnerable and underserved populations in the community? 

If we had point data of where these medical service providers and clinics were located, we could look for where there are gaps in service provision (high vulnerability and no providers) that could help guide future provider site selection and outreach.

Read in manually scraped community health partners excel sheet to map

In [ ]:
#Coming from excel sheet we created, setting index_column at 0, and specifying what datatype we want
#specific columns in

com_partners = pd.read_excel('../data/dell_med-community_partners_geo.xlsx', index_col=0,
                             dtype={'name': str, 'address': str, 'provider':str,'lat-lon':str,'sub-type':str})

In [ ]:
com_partners

In [ ]:
#We need to take the lat-lon column, break it into separate lat and lon columns in preperation for turning them 
#into geometries for mapping

lat = []
lon = []

for x in com_partners['lat-lon']:
    y = x.split(", ")
    lat.append(y[0])
    lon.append(y[1])
    
com_partners['lat']= lat
com_partners['lon']= lon

com_partners

In [ ]:
#Make a geometry column from the the dataframe from the geo_ident 'lon' and 'lat columns'
#This will give you the coordinates of each community saved as a point type of geometry which can be plotted in GIS

com_partners_geo = gpd.GeoDataFrame(
    com_partners, geometry=gpd.points_from_xy(com_partners.lon, com_partners.lat,crs=4269))

com_partners_geo

Make a point-data map of the community partners data using the same plot function and arguments from above

In [ ]:
com_partners_geo.plot(column=com_partners_geo['provider'], markersize=40, legend=True, figsize=(15,10),
                      categorical=True, cmap='Paired')

While this is interesting to see the spread of these points, there's no other geographic information in this layer to help us devleop any insights from the spatial distribution of these community partners. This is where adding multiple layers to a map comes in.

If we could add in a raster basemap layer (think google maps) under these points, we could see where these points are located relative to highways, citieis and neighborhoods in Austin, TX. 

To add a baselayer, we will use the contextily package we loaded in earlier as cx. Contextily has a whole library of different baselayers and label styles we can draw from; however, it is important to know that all these baselayers are set in a crs or projection of 3857. So any layers we want to add cx baselayers to need to be in the same crs.   

In [ ]:
#Set the crs for this point layer to 3857 so it works with contextily basemaps
com_partners_geo = com_partners_geo.to_crs(epsg=3857)

partners = com_partners_geo.plot(column=com_partners_geo['provider'],zorder=3, markersize=120, legend=True, 
                                 figsize=(15,10),
                                 categorical=True, cmap='Paired', edgecolor='black')

#USe contextily .add_basemap function to call in basemaps, adding them to the partners plot, and setting their 
#"zoom" level
cx.add_basemap(partners,zorder=1,zoom=11, source=cx.providers.Stamen.Toner)
cx.add_basemap(partners,zorder=2,zoom=1, source=cx.providers.Stamen.TonerLabels)

partners.axis('off')

__Exercise #3__: Try changing around the zorder of the map layers and see what happens. Play around with the zoom level as well to see exactly what zoom does and find the best level of zoom for making this map visually useful. 

Change the baselayer and label type as well using the following code:
- cx.add_basemap(ax,zorder=1,zoom=11, source=cx.providers.Stamen.Watercolor)
- cx.add_basemap(ax,zorder=1,zoom=10, source=cx.providers.CartoDB.Voyager)

- cx.add_basemap(ax,zorder=1,zoom=11, source=cx.providers.Stamen.WatercolorLabels)
- cx.add_basemap(ax,zorder=1,zoom=10, source=cx.providers.CartoDB.VoyagerLabels)

In [ ]:
#Exercise #3 Code




Now let's add the SVI census tract layer from above to see whether these community partners are located in areas of high social vulnerability, their stated mission. 

In [ ]:
five_counties = five_counties.to_crs(epsg=3857)
five_county_shp = five_county_shp.to_crs(epsg=3857)
com_partners_geo = com_partners_geo.to_crs(epsg=3857)

plt.figure()

base = five_counties.plot(column='F_TOTAL',zorder=2, figsize=(20, 15),cmap='OrRd',scheme='natural_breaks', 
                          edgecolor='black',
                          legend=True,
                         missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
                       "alpha":0.2,
        "hatch": "///",
        "label": "Missing values"})

full = five_county_shp.plot(ax=base,zorder=3,facecolor='none',edgecolor='black',linewidth=3)

partners = com_partners_geo.plot(ax=full,zorder=5, column=com_partners_geo['provider'], markersize=80, legend=True,
                                 categorical=True, cmap='Paired', edgecolor='black')

full.set_axis_off()
plt.suptitle('Social Vulnerability Index for Central Texas Five-County Region',y=.93, fontsize=25)
plt.title('Total Vulnerability Flags by Census Tract and Plotted Community Partners')

cx.add_basemap(partners,zorder=1,zoom=11, source=cx.providers.Stamen.Toner)
cx.add_basemap(partners,zorder=4,zoom=10, source=cx.providers.Stamen.TonerLabels)

ax.axis('off')

#plt.savefig('../images/ct5_svi_total_flags_cpartners_basemap.jpg')

What insights can we glean from this map?